In [1]:
from binance import Client
import pandas as pd

In [ ]:
key = your_key
secret = your_secret

In [ ]:
client = Client(key, secret) 

In [19]:
posframe = pd.read_csv('positions')


In [4]:
posframe

,Currency,position,quantity
0,BTCUSDT,0,0.002
1,ETHUSDT,0,0.020
2,ADAUSDT,0,100.000


In [21]:
def changepos(curr, buy = True):
    if buy:
        posframe.loc[posframe.Currency == curr, 'positions'] = 1
    else:
        posframe.loc[posframe.Currency == curr, 'positions'] = 0 
    
    posframe.to_csv('positions', index = False)


In [32]:
def gethourlydata(symbol):
    frame = pd.DataFrame(Client.get_historical_klines(symbol, '1h', '25 hours ago UTC'))
    frame = frame.iloc[:,:5]
    frame.columns = ['Time','Open','High','Low','Close']
    frame[['Open','High','Low','Close']] = frame[['Open','High','Low','Close']].astype(float)
    frame.Time = pd.to_datetime(frame.Time, unit = 'ms')
    return frame


In [ ]:
df = gethourlydata('BTCUSDT')

In [26]:
def applytechnicals(df):
    df['FastSMA'] = df.Close.rolling(7).mean()
    df['SlowSMA'] = df.Close.rolling(25).mean()

In [ ]:
applytechnicals(df)

In [ ]:
def trader(curr):
    qty = posframe[posframe.Currency == curr].quantity.values[0]
    df = gethourlydata(curr)
    applytechnicals(df)
    lastrow = df.iloc[-1]
    if not posframe[posframe.Currency == curr].position.values[0]:
        if lastrow.FastSMA > lastrow.SlowSMA:
            order = Client.create_order(symbol = curr, side = 'BUY', type = 'MARKET', quantity = qty)

            print(order)
            changepos(curr, buy = True)
        else:
            print(f'Not in position {curr} but Condition not fulfilled')
    
    else:
        print(f'already in {curr} position')
        if lastrow.SlowSMA > lastrow.FastSMA:
            order = client.create_order(symbol=curr,side = 'SELL', type='MARKET', quantity = qty)
            print(order)
            changepos(curr, buy=False)


In [ ]:
for coin in posframe.Currency:
    trader(coin)